In [0]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

batch_size = 45  # Batch size for training.
epochs = 700  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'recaps-mod.txt'

In [0]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
count = 1 
with open(data_path, 'r', encoding='utf-8') as f:
  lines = f.read().split('\n')
  for line in lines[: min(num_samples, len(lines) - 1)]:
      input_text, target_text = line.split('\t')
      # We use "tab" as the "start sequence" character
      # for the targets, and "\n" as "end sequence" character.
      target_text = '\t' + target_text + '\n'
      input_texts.append(input_text)
      target_texts.append(target_text)
      for char in input_text:
          if char not in input_characters:
              input_characters.add(char)
      for char in target_text:
          if char not in target_characters:
              target_characters.add(char)

In [53]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)


Number of samples: 179
Number of unique input tokens: 63
Number of unique output tokens: 65
Max sequence length for inputs: 75
Max sequence length for outputs: 128


In [0]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [0]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.


In [0]:
from keras.layers.embeddings import Embedding

In [57]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')

Train on 143 samples, validate on 36 samples
Epoch 1/700
143/143 [==============================] - 4s 31ms/step - loss: 3.1768 - val_loss: 2.9933
Epoch 2/700
143/143 [==============================] - 2s 15ms/step - loss: 2.6212 - val_loss: 2.7623
Epoch 3/700
143/143 [==============================] - 2s 15ms/step - loss: 2.5195 - val_loss: 2.7716
Epoch 4/700
143/143 [==============================] - 2s 15ms/step - loss: 2.5124 - val_loss: 2.7445
Epoch 5/700
143/143 [==============================] - 2s 15ms/step - loss: 2.4969 - val_loss: 2.7198
Epoch 6/700
143/143 [==============================] - 2s 16ms/step - loss: 2.4731 - val_loss: 2.7127
Epoch 7/700
143/143 [==============================] - 2s 15ms/step - loss: 2.4619 - val_loss: 2.6893
Epoch 8/700
143/143 [==============================] - 2s 15ms/step - loss: 2.4615 - val_loss: 2.7110
Epoch 9/700
143/143 [==============================] - 2s 16ms/step - loss: 2.4459 - val_loss: 2.6927
Epoch 10/700
143/143 [===============

/usr/local/lib/python3.6/dist-packages/keras/engine/network.py:877: UserWarning: Layer lstm_6 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_5/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_5/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


In [0]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [0]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [60]:
print("Here we validate our training results")

for seq_index in range(50):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)
    


Here we validate our training results
-
Input sentence: Willie Cauley-Stein, San Antonio Spurs, 51 points, 10 assists, 12 rebounds.
Decoded sentence: Willie Cauley-Stein played a great game tonight against the San Antonio Spurs with 51 points, 10 assists, and 12 rebounds.

-
Input sentence: Marvin Bagley, Detroit Pistons, 31 points, 10 assists, 12 rebounds.
Decoded sentence: Marvin Bagley played a great game tonight against the Detroit Pistons with 31 points, 10 assists, and 12 rebounds.

-
Input sentence: Michael Beasley, New Orleans Pelicans, 51 points, 4 assists, 11 rebounds.
Decoded sentence: Michael Beasley played a great game tonight against the New Orleans Pelicans with 51 points, 4 assists, and 11 rebounds.

-
Input sentence: Jose Calderon, Indiana Pacers, 40 points, 13 assists, 5 rebounds.
Decoded sentence: Jose Calderon played a great game tonight against the Indiana Pacers with 40 points, 13 assists, and 5 rebounds.

-
Input sentence: Wayne Ellington, Los Angles Lakers, 40 p

In [65]:

def output():
  mylist_in = [0] * 100
  mylist_out = [0] * 100
  count = 0
  for seq_index in range(1, 2):
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    mylist_in[count] = input_texts[seq_index]
    mylist_out[count] = decoded_sentence
    count += 1 
  for seq_index in range(32, 33):
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    mylist_in[count] = input_texts[seq_index]
    mylist_out[count] = decoded_sentence
    count += 1 
    
    for seq_index in range(29, 30):
      input_seq = encoder_input_data[seq_index: seq_index + 1]
      decoded_sentence = decode_sequence(input_seq)
      mylist_in[count] = input_texts[seq_index]
      mylist_out[count] = decoded_sentence
      count += 1 
    
  for seq_index in range(18, 19):
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    mylist_in[count] = input_texts[seq_index]
    mylist_out[count] = decoded_sentence
    count += 1 
  return mylist_in, mylist_out, count



a1, a2, a3 = output()
import sys

print("Here we pass our stats")
print()

for x in range(0,a3):
  print(a1[x])
  
print()
print("+++++++++++++++++++++")
print()
  
print("Here we print our smart recaps")
print()

for x in range(0,a3):
  sys.stdout.write(a2[x])
  

print()
print("Generating an additional stat")
  
def output():
  mylist_in = [0] * 100
  mylist_out = [0] * 100
  count = 0
  for seq_index in range(60, 61):
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    mylist_in[count] = input_texts[seq_index]
    mylist_out[count] = decoded_sentence
    count += 1 
  for seq_index in range(32, 33):
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    mylist_in[count] = input_texts[seq_index]
    mylist_out[count] = decoded_sentence
    count += 1 
    
    for seq_index in range(29, 30):
      input_seq = encoder_input_data[seq_index: seq_index + 1]
      decoded_sentence = decode_sequence(input_seq)
      mylist_in[count] = input_texts[seq_index]
      mylist_out[count] = decoded_sentence
      count += 1 
    
  for seq_index in range(15, 16):
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    mylist_in[count] = input_texts[seq_index]
    mylist_out[count] = decoded_sentence
    count += 1 
  return mylist_in, mylist_out, count



a1, a2, a3 = output()
import sys

print("Here we pass our stats")
print()

for x in range(0,a3):
  print(a1[x])
  
print()
print("+++++++++++++++++++++")
print()
  
print("Here we print our smart recaps")
print()

for x in range(0,a3):
  sys.stdout.write(a2[x])
  






Here we pass our stats

Marvin Bagley, Detroit Pistons, 31 points, 10 assists, 12 rebounds.
OS.
OV.
Detroit Pistons, Miami Heat, 99, 81.

+++++++++++++++++++++

Here we print our smart recaps

Marvin Bagley played a great game tonight against the Detroit Pistons with 31 points, 10 assists, and 12 rebounds.
This game started off slow, but sure enough it started picking up in the second half.
Both teams played a fantastic game, and hustled extremely hard giving the crowd exactly what they were looking for. 
Detroit Pistons had a great game against Miami Heat -- 99 - 81.

Generating an additional stat
Here we pass our stats

Kyle Korver 37 points, 3 assists, 12 rebounds, Indiana Pacers.
OS.
OV.
Indiana Pacers, Utah Jazz, 117, 92.

+++++++++++++++++++++

Here we print our smart recaps

Beautiful game last night for Kyle Korver who had 37 points, 3 assists, and 12 rebounds against the Indiana Pacers.
This game started off slow, but sure enough it started picking up in the second half.
Both 